In [44]:
pip install pyth


  Preparing metadata (setup.py) ... done
  Created wheel for pyth: filename=pyth-0.6.0-py3-none-any.whl size=27065 sha256=48a9971a80752dc75e1d702b43087991c695f25c16d9750ab9069010b5da0af1
  Stored in directory: /root/.cache/pip/wheels/96/f2/93/9dd09c802457fc6bdced874be37c67d884eb8adb82365d0179
Successfully built pyth


In [40]:
pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install striprtf

In [1]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.0 MB/s eta 0:00:00


In [1]:
import random
import pandas as pd
from transformers import GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
from torch.nn.functional import cosine_similarity


def generate_synthetic_data(num_samples=1000):
    words = ["apple", "banana", "orange", "grape", "pineapple"]
    data = []
    for _ in range(num_samples):
        sentence_length = random.randint(5, 10)
        sentence = " ".join(random.choices(words, k=sentence_length))
        data.append(sentence)
    return data

synthetic_data = generate_synthetic_data()
df = pd.DataFrame(synthetic_data, columns=['text'])
df.to_csv("synthetic_data.csv", index=False)


In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Tokenize and encode the synthetic dataset
def encode(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=50)

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="synthetic_data.csv",
    block_size=50
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-synthetic",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=230, training_loss=1.6916080640709918, metrics={'train_runtime': 24.9116, 'train_samples_per_second': 36.529, 'train_steps_per_second': 9.233, 'total_flos': 23220288000000.0, 'train_loss': 1.6916080640709918, 'epoch': 5.0})

In [3]:
trainer.save_model("./gpt2-synthetic-finetuned")


In [4]:
from transformers import GPT2Model, GPT2Config
import torch
import torch.nn as nn



In [11]:
class CustomAttentionGPT2(GPT2LMHeadModel):
    def __init__(self, config):
        super().__init__(config)
        self.attention_bias = nn.Parameter(torch.zeros(config.n_positions, config.n_positions))

    def forward(self, input_ids, **kwargs):
        outputs = super().forward(input_ids, output_attentions=True, **kwargs)

        attention_scores = outputs.attentions
        pattern_ids = [tokenizer.encode(pattern, add_special_tokens=False) for pattern in ["apple", "banana"]]  # Example patterns
        bias = self.compute_bias(input_ids, pattern_ids)
        seq_length = input_ids.size(1)
        bias = bias[:seq_length, :seq_length].unsqueeze(0).unsqueeze(0)  # New shape: [1, 1, seq_length, seq_length]
        attention_scores = [score + bias for score in attention_scores]

        outputs = outputs[:1] + (attention_scores,) + outputs[2:]
        return outputs

    def compute_bias(self, input_ids, pattern_ids):
        bias = torch.zeros_like(self.attention_bias)
        for pattern_id in pattern_ids:
            for i, token_id in enumerate(input_ids.squeeze().tolist()):
                if token_id in pattern_id:
                    bias[i, :] += 1.0
        return bias

model = CustomAttentionGPT2(config)
input_ids = tokenizer.encode("I like apples and bananas", return_tensors="pt")
outputs = model(input_ids)


In [28]:
outputs[1][0].shape
"""In the case of GPT-2, which has 12 layers, the outputs.attentions list will contain 12
tensors, each with a shape of [batch_size, num_heads, sequence_length, sequence_length].
Therefore, in your code, attention_scores will be a list of 12 tensors, each with a
 shape of [1, 12, 5, 5]."""


'In the case of GPT-2, which has 12 layers, the outputs.attentions list will contain 12 \ntensors, each with a shape of [batch_size, num_heads, sequence_length, sequence_length]. \nTherefore, in your code, attention_scores will be a list of 12 tensors, each with a\n shape of [1, 12, 5, 5].'

In [29]:

class CustomAttentionGPT2(GPT2LMHeadModel):
    def __init__(self, config):
        super().__init__(config)
        self.attention_bias = nn.Parameter(torch.zeros(config.n_positions, config.n_positions))

    def forward(self, input_ids, **kwargs):
        outputs = super().forward(input_ids, output_attentions=True, **kwargs)

        attention_scores = outputs.attentions
        pattern_ids = [tokenizer.encode(pattern, add_special_tokens=False) for pattern in ["apple", "banana"]]  # Example patterns
        bias = self.compute_bias(input_ids, pattern_ids)
        seq_length = input_ids.size(1)
        bias = bias[:seq_length, :seq_length].unsqueeze(0).unsqueeze(0)  # New shape: [1, 1, seq_length, seq_length]
        attention_scores = [score + bias for score in attention_scores]

        outputs = outputs[:1] + (attention_scores,) + outputs[2:]
        return outputs

    def compute_bias(self, input_ids, pattern_ids):
        bias = torch.zeros_like(self.attention_bias)
        for pattern_id in pattern_ids:
            for i, token_id in enumerate(input_ids.squeeze().tolist()):
                if token_id in pattern_id:
                    bias[i, :] += 1.0
        return bias

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

config = GPT2Config.from_pretrained("gpt2")
model = CustomAttentionGPT2(config)

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="synthetic_data.csv",
    block_size=50
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="./gpt2-synthetic",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=230, training_loss=2.380313773777174, metrics={'train_runtime': 23.806, 'train_samples_per_second': 38.226, 'train_steps_per_second': 9.661, 'total_flos': 23506549248000.0, 'train_loss': 2.380313773777174, 'epoch': 5.0})

In [30]:
import torch
from torch.nn.functional import cosine_similarity
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Model

class CustomAttentionGPT2(GPT2LMHeadModel):
    def __init__(self, config, embeddings, similarity_threshold):
        super().__init__(config)
        self.attention_bias = torch.nn.Parameter(torch.zeros(config.n_positions, config.n_positions))
        self.embeddings = embeddings
        self.similarity_threshold = similarity_threshold

    def forward(self, input_ids, **kwargs):
        outputs = super().forward(input_ids, output_attentions=True, **kwargs)

        attention_scores = outputs.attentions
        pattern_ids = [tokenizer.encode(pattern, add_special_tokens=False) for pattern in ["apple", "banana"]]  # Example patterns
        bias = self.compute_bias(input_ids, pattern_ids)
        seq_length = input_ids.size(1)
        bias = bias[:seq_length, :seq_length].unsqueeze(0).unsqueeze(0)  # New shape: [1, 1, seq_length, seq_length]
        attention_scores = [score + bias for score in attention_scores]

        outputs = outputs[:1] + (attention_scores,) + outputs[2:]
        return outputs

    def compute_bias(self, input_ids, pattern_ids):
        bias = torch.zeros_like(self.attention_bias)
        input_ids = input_ids.squeeze().to(self.embeddings.device)  # Ensure input_ids are on the same device as embeddings
        input_embeddings = self.embeddings[input_ids]  # Index embeddings with input_ids

        for pattern_id in pattern_ids:
            pattern_embedding = self.embeddings[pattern_id].mean(dim=0)  # Average embedding for the pattern
            for i, token_embedding in enumerate(input_embeddings):
                # Compute cosine similarity for each element
                similarity = cosine_similarity(token_embedding.unsqueeze(0), pattern_embedding.unsqueeze(0), dim=1)
                # Check if any similarity value meets the threshold
                if (similarity >= self.similarity_threshold).any():
                    bias[i, :] += similarity.max()  # Use the maximum similarity value

        return bias



pretrained_model = GPT2Model.from_pretrained("gpt2")

# Extract the embeddings from the pretrained model
embeddings = pretrained_model.get_input_embeddings().weight.detach()

# Example usage
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
config = GPT2Config.from_pretrained("gpt2")

# Create an instance of your custom model with the embeddings and similarity threshold
similarity_threshold = 0.5  # Set your desired similarity threshold
model = CustomAttentionGPT2(config, embeddings, similarity_threshold)

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="synthetic_data.csv",
    block_size=50
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="./gpt2-synthetic",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()


Step,Training Loss


TrainOutput(global_step=230, training_loss=2.2973898182744565, metrics={'train_runtime': 25.4247, 'train_samples_per_second': 35.792, 'train_steps_per_second': 9.046, 'total_flos': 23506549248000.0, 'train_loss': 2.2973898182744565, 'epoch': 5.0})

In [31]:
model = CustomAttentionGPT2(config,  embeddings , similarity_threshold)
input_ids = tokenizer.encode("I like apples and bananas", return_tensors="pt")
outputs = model(input_ids)


In [34]:
outputs[1][0].shape

torch.Size([1, 12, 5, 5])

In [54]:
!pip install striprtf

In [ ]:
# Open the file in read mode
with open('/content/syntheticCausal.csv', 'r') as file:
    # Read the entire file contents
    content = file.read()
    print(content)


In [58]:
content


"\t\tThe heavy rain caused the river to overflow.\n\t\tEating too much sugar can lead to obesity.\n\t\tThe earthquake triggered a massive tsunami.\n\t\tLack of sleep often results in poor concentration.\n\t\tRegular exercise contributes to better health.\n\t\tThe government's policies led to economic growth.\n\t\tHigh temperatures caused the ice to melt.\n\t\tSmoking increases the risk of lung cancer.\n\t\tThe new law resulted in a decrease in crime.\n\t\tPollution is a major cause of global warming.\n\t\tA balanced diet can prevent many diseases.\n\t\tThe storm caused widespread power outages.\n\t\tStress often leads to health problems.\n\t\tVaccination has led to the eradication of many diseases.\n\t\tOverfishing causes a decline in fish populations.\n\t\tDeforestation results in loss of habitat for wildlife.\n\t\tGood communication can prevent misunderstandings.\n\t\tThe discovery of penicillin revolutionized medicine.\n\t\tPoor hygiene can lead to the spread of infections.\n\t\tA s